# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298630014534                   -0.85    5.2
  2   -8.300224127516       -2.80       -1.25    1.1
  3   -8.300451005630       -3.64       -1.89    2.4
  4   -8.300463323758       -4.91       -2.78    2.4
  5   -8.300464500301       -5.93       -3.14    3.9
  6   -8.300464576912       -7.12       -3.28    1.5
  7   -8.300464610707       -7.47       -3.41    4.0
  8   -8.300464628794       -7.74       -3.55    1.0
  9   -8.300464639892       -7.95       -3.73    1.0
 10   -8.300464642289       -8.62       -3.88    1.8
 11   -8.300464643792       -8.82       -4.22    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67468780521                   -0.70    6.6
  2   -16.67869991657       -2.40       -1.14    1.4
  3   -16.67923552573       -3.27       -1.87    3.4
  4   -16.67928104907       -4.34       -2.77    3.4
  5   -16.67928604269       -5.30       -3.22    5.2
  6   -16.67928621919       -6.75       -3.52    4.5
  7   -16.67928622338       -8.38       -3.98    1.0
  8   -16.67928622426       -9.06       -4.63    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32572429047                   -0.56    7.5
  2   -33.33273045351       -2.15       -1.00    1.0
  3   -33.33412792515       -2.85       -1.74    5.2
  4   -33.33428572156       -3.80       -2.64    2.5
  5   -33.33694302065       -2.58       -2.52   12.9
  6   -33.33694338376       -6.44       -2.54    1.0
  7   -33.33694359819       -6.67       -3.07    1.4
  8   -33.33694378146       -6.74       -3.60    4.6
  9   -33.33694392380       -6.85       -3.80    1.8
 10   -33.33694392663       -8.55       -4.49    2.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298297457134                   -0.85    5.0
  2   -8.300235788899       -2.71       -1.59    1.0
  3   -8.300422839257       -3.73       -2.39    5.0
  4   -8.300316039244   +   -3.97       -2.17   14.0
  5   -8.300464406122       -3.83       -3.60    2.8
  6   -8.300464593584       -6.73       -3.95    3.0
  7   -8.300464638679       -7.35       -4.41    2.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32388727458                   -0.56    6.6
  2   -33.29754729120   +   -1.58       -1.26    1.8
  3   +1.297503689512   +    1.54       -0.30    9.0
  4   -33.32132435663        1.54       -1.63    7.6
  5   -33.24240474325   +   -1.10       -1.32    5.6
  6   -32.85820384352   +   -0.42       -1.22    4.4
  7   -33.28288097095       -0.37       -1.69    5.0
  8   -33.33666329752       -1.27       -2.29    3.1
  9   -33.33685591244       -3.72       -2.74    3.2
 10   -33.33687643286       -4.69       -2.78    3.5
 11   -33.33692246675       -4.34       -2.96    1.8
 12   -33.33693643955       -4.85       -3.18    2.1
 13   -33.33694271145       -5.20       -3.60    3.0
 14   -33.33694339567       -6.16       -3.98    2.6
 15   -33.33694388156       -6.31       -4.53    3.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.